In [224]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [225]:
model=load_model('model.h5')
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)    

c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [226]:
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProduct' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [227]:
geo_encoded = onehot_encoder_geo.transform([['France']]).toarray()  # Note double brackets
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [228]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProduct,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [229]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProduct,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [230]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProduct,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [231]:
# Rename the column to match what the scaler expects
input_df = input_df.rename(columns={'NumOfProduct': 'NumOfProducts'})

# Ensure all columns are present and in correct order
input_df = input_df[scaler.feature_names_in_]

# Now transform
input_scaled = scaler.transform(input_df)

In [232]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [233]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 67ms/step


array([[0.03069633]], dtype=float32)

In [234]:
prediction_proba = prediction[0][0]

In [235]:
prediction_proba

0.03069633

In [237]:
if prediction_proba > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to chrun.')

The customer is not likely to chrun.
